In [1]:
import os
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from tqdm import tqdm_notebook as tqdm
from torch import nn
import copy
import transformers
print(transformers.__version__)
from transformers.modeling_t5 import *
import json
import numpy as np
import pickle
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from transformers import *
import torch
# import matplotlib.pyplot as plt
import torch.utils.data
import torch.nn.functional as F
from torch.optim import Adagrad, Adamax
from transformers.modeling_utils import *
from scipy.special import softmax
import argparse
import itertools
from collections import OrderedDict
from fuzzywuzzy import fuzz
import operator
from models import *
from transformers.tokenization_bert import BasicTokenizer
from tqdm import tqdm_notebook as tqdm

2.8.0


In [2]:
batch_size = 64
beam_size = 3
max_sequence_length = 128

In [15]:
def find_best_combinations(start_top_log_probs, start_top_index, end_top_log_probs, end_top_index, valid_start= 0, valid_end=512):
    best = (valid_start, valid_end - 1)
    best_score = -9999
    for i in range(len(start_top_log_probs)):
        for j in range(end_top_log_probs.shape[0]):
            if valid_start <= start_top_index[i] < valid_end and valid_start <= end_top_index[j,i] < valid_end and start_top_index[i] < end_top_index[j,i]:
                score = start_top_log_probs[i] * end_top_log_probs[j,i]
                if score > best_score:
                    best = (start_top_index[i],end_top_index[j,i])
                    best_score = score
    return best


def jaccard(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    c = a.intersection(b)
    try:
        return float(len(c)) / (len(a) + len(b) - len(c))
    except:
        return 0
    
# def fuzzy_match(x,y):
#     l1,l2 = len(x.split()), len(y.split())
#     matches = dict()
#     x_ = x.split()
#     for curr_length in range(l1 + 1):
#         for i in range(l1 + 1 - curr_length):
#             sub_x = ' '.join(x_[i:i+curr_length])
#             if sub_x not in matches:
#                 matches[sub_x] = fuzz.ratio(sub_x,y)
#     if len(matches) == 0:
#         return None, x
#     best_score = np.max([val for val in matches.values()])
#     best_matches = [key for key,val in matches.items() if val == best_score]
#     if len(best_matches) == 1:
#         return matches, sorted(matches.items(), key=operator.itemgetter(1))[-1][0]
#     else:
#         return matches, best_matches[np.argmax([len(m.split()) for m in best_matches])]

def fuzzy_match(x,y,weights=None):
    l1 = len(x.split())
    matches = dict()
    x_ = x.split()
    if type(y) is str:
        y = [y]
    for curr_length in range(l1 + 1):
        for i in range(l1 + 1 - curr_length):
            sub_x = ' '.join(x_[i:i+curr_length])
            if sub_x not in matches:
                matches[sub_x] = np.average([fuzz.ratio(sub_x,y_) for y_ in y],weights=weights)
    if len(matches) == 0:
        return None, x
    return matches, sorted(matches.items(), key=operator.itemgetter(1))[-1][0]
    
def ensemble_v0(context, predictions):
    context = context.split()
    scores = dict()
    for i,j in itertools.combinations(range(len(context) + 1),r=2):
        curr_context = ' '.join(context[i:j])
        scores[curr_context] = np.mean([jaccard(curr_context, p) for p in predictions])
    best_score = np.max([val for val in scores.values()])
    has_tie = np.sum([val == best_score for val in scores.values()]) > 1
    if not has_tie:
        for key, val in scores.items():
            if val == best_score:
                return key
    else:
        keys = [key for key, val in scores.items() if val == best_score]
#         return keys[np.argmax([jaccard(key,predictions[0]) for key in keys])]
        return keys[np.argmax([len(key.split()) for key in keys])]

def ensemble(context, predictions):
    starts = []
    ends = []
    for p in predictions:
        if p in context:
            start = context.index(p)
            starts.append(start)
            ends.append(start+len(p))
    if len(starts) == 0:
        print(context)
        return ensemble_v0(context, predictions)
    scores = dict()
    context = context[np.min(starts):np.max(ends)]
    for i,j in itertools.combinations(range(len(context) + 1),r=2):
        if len(context.split()) == 1 or \
            (not ((i > 0 and context[i-1].isalnum() and context[i].isalnum()) or (j < len(context) and j > 1 and context[j-1].isalnum() and context[j].isalnum()))):
            curr_context = context[i:j].strip()
            scores[curr_context] = np.mean([jaccard(curr_context, p) for p in predictions])
    for pred in predictions:
        if pred not in scores:
            scores[pred] = np.mean([jaccard(pred, p) for p in predictions])
    best_score = np.max([val for val in scores.values()])
    has_tie = np.sum([val == best_score for val in scores.values()]) > 1
    if not has_tie:
        for key, val in scores.items():
            if val == best_score:
                return key
    else:
        keys = [key for key, val in scores.items() if val == best_score]
#         return keys[np.argmax([jaccard(key,predictions[0]) for key in keys])]
        return keys[np.argmax([len(key.split()) for key in keys])]

basic_tokenizer = BasicTokenizer(do_lower_case=False)
def fix_spaces(t):
    for i,item in enumerate(t):
        re_res = re.search('\s+$', item)
        if bool(re_res) & (i < len(t)-1):
            sp = re_res.span()
            t[i+1] = t[i][sp[0]:] + t[i+1]
            t[i] = t[i][:sp[0]]
    return t
def roberta_tokenize_v2(tokenizer, line):
    tokenized_line = []
    line2 = basic_tokenizer._run_split_on_punc(line)
    line2 = fix_spaces(line2)
    for item in line2:
        sub_word_tokens = tokenizer.tokenize(item)
        tokenized_line += sub_word_tokens
    return tokenized_line


def convert_lines(tokenizer, df, max_sequence_length = 512):
    pad_token_idx = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
    outputs = np.zeros((len(df), max_sequence_length))
    type_outputs = np.zeros((len(df), max_sequence_length))
    position_outputs = np.zeros((len(df), 2))
    extracted = []
    for idx, row in tqdm(df.iterrows(), total=len(df)): 
        input_ids_0 = tokenizer.convert_tokens_to_ids(roberta_tokenize_v2(tokenizer, row.sentiment)) 
        input_ids_1 = tokenizer.convert_tokens_to_ids(roberta_tokenize_v2(tokenizer, row.text)) 
        input_ids = [tokenizer.cls_token_id, ]+ input_ids_0 +  [tokenizer.sep_token_id,] +input_ids_1 + [tokenizer.sep_token_id, ]
        token_type_ids = [0,]*(len(input_ids_0) + 1) + [1,]*(len(input_ids_1) + 2)
        if len(input_ids) > max_sequence_length: 
#            input_ids = input_ids[:max_sequence_length//2] + input_ids[-max_sequence_length//2:] 
            input_ids = input_ids[:max_sequence_length]
            input_ids[-1] = tokenizer.sep_token_id
            token_type_ids = token_type_ids[:max_sequence_length]
        else:
            input_ids = input_ids + [pad_token_idx, ]*(max_sequence_length - len(input_ids))
            token_type_ids = token_type_ids + [pad_token_idx, ]*(max_sequence_length - len(token_type_ids))
        assert len(input_ids) == len(token_type_ids)
        outputs[idx,:max_sequence_length] = np.array(input_ids)
        type_outputs[idx,:] = token_type_ids
    return outputs, type_outputs

In [4]:
# test_df = pd.read_csv("./data/test_holdout.csv")
test_df = pd.read_csv("./data/test.csv")

# splits = list(KFold(n_splits=5, shuffle=True, random_state=53).split(test_df, np.arange(len(test_df))))
# test_df = test_df.loc[splits[0][1]].reset_index()
test_df["sep_text"] = test_df.text.apply(lambda x: " ".join(x.split()).lower())
def get_predictions(x_test, x_type_test, model, is_xlnet=False):
    all_start_top_log_probs = None
    all_start_top_index = None
    all_end_top_log_probs = None
    all_end_top_index = None
    test_dataset = torch.utils.data.TensorDataset(torch.tensor(x_test,dtype=torch.long), torch.tensor(x_type_test,dtype=torch.long))
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        pbar = tqdm(enumerate(test_loader),total=len(test_loader),leave=False)
        for i, items in pbar:
            x_batch, x_type_batch = items
            attention_mask = x_batch != tokenizer.pad_token_id
            p_mask = torch.zeros(x_batch.shape,dtype=torch.float32)
            p_mask[x_batch == tokenizer.pad_token_id] = 1.0
            p_mask[x_batch == tokenizer.cls_token_id] = 1.0
            if is_xlnet:
                attention_mask = attention_mask.float()
                p_mask[:,:2] = 1.0
            else:
                p_mask[:,:3] = 1.0
            start_top_log_probs, start_top_index, end_top_log_probs, end_top_index = model(input_ids=x_batch.cuda(), attention_mask=attention_mask.cuda(), \
                                                token_type_ids=x_type_batch.cuda(), beam_size=beam_size, p_mask=p_mask.cuda())
            start_top_log_probs = start_top_log_probs.detach().cpu().numpy()
            start_top_index = start_top_index.detach().cpu().numpy()
            end_top_log_probs = end_top_log_probs.detach().cpu().numpy()
            end_top_index = end_top_index.detach().cpu().numpy()

            all_start_top_log_probs = start_top_log_probs if all_start_top_log_probs is None else np.concatenate([all_start_top_log_probs, start_top_log_probs])
            all_start_top_index = start_top_index if all_start_top_index is None else np.concatenate([all_start_top_index, start_top_index])
            all_end_top_log_probs = end_top_log_probs if all_end_top_log_probs is None else np.concatenate([all_end_top_log_probs, end_top_log_probs])
            all_end_top_index = end_top_index if all_end_top_index is None else np.concatenate([all_end_top_index, end_top_index])

    return all_start_top_log_probs,all_start_top_index,all_end_top_log_probs,all_end_top_index

def load_and_fix_state(model_path):
    state_dict = torch.load(model_path)
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
    return new_state_dict

def get_selected_texts(x_test, tokenizer, offset=3, is_xlnet=False):
    selected_texts = []
    for i_, x in tqdm(enumerate(x_test),total=len(x_test)):
        real_length = np.sum(x != tokenizer.pad_token_id)
        if is_xlnet:
            real_length -= 1
        best_start, best_end = find_best_combinations(all_start_top_log_probs[i_], all_start_top_index[i_], \
                                                        all_end_top_log_probs[i_].reshape(beam_size,beam_size), all_end_top_index[i_].reshape(beam_size,beam_size), \
                                                        valid_start = offset, valid_end = real_length)
#         selected_text = tokenizer.decode([w for w in x[best_start:best_end] if w != tokenizer.pad_token_id]).strip()
#         selected_text = " ".join(selected_text.split()).lower()
#         selected_texts.append(selected_text.lower() if selected_text in test_df.loc[i_].sep_text 
#                               else fuzzy_match(test_df.loc[i_].sep_text, selected_text)[-1])
        selected_text = tokenizer.decode([w for w in x[best_start:best_end] if w != tokenizer.pad_token_id],clean_up_tokenization_spaces=False)
        selected_texts.append(selected_text if selected_text in test_df.loc[i_].text 
                              else fuzzy_match(test_df.loc[i_].text, selected_text)[-1])
    return selected_texts

def check_corrs(model_name):
    corrs = np.zeros((5,5))
    for i in range(5):
        for j in range(5):
            if corrs[j,i] == 0:
                corrs[i,j] = np.mean([jaccard(x,y) for x,y in zip(all_preds[f"{model_name}_{i}"], all_preds[f"{model_name}_{j}"])])
                corrs[j,i] = corrs[i,j]
    return corrs

In [5]:
all_preds = dict()

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSentimentExtraction.from_pretrained('roberta-base', output_hidden_states=True)
model.cuda()
X_test, X_type_test = convert_lines(tokenizer, test_df, max_sequence_length= max_sequence_length)
seeds = [13,23,33]
for seed in seeds:
    for fold in range(5):
        model.load_state_dict(torch.load(f"./models/roberta_{fold}_{seed}.bin"), strict=False)
        model.eval()
        all_start_top_log_probs,all_start_top_index,all_end_top_log_probs,all_end_top_index = get_predictions(X_test, X_type_test, model)
        selected_texts = get_selected_texts(X_test, tokenizer)
        print(selected_texts[:10])
        all_preds[f"roberta_{fold}_{seed}"] = [x if type(x) is str else "" for x in selected_texts]


['Last session of the day', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']


...


['Last session of the day', ' exciting', ' such a shame!', ' happy bday!', ' like', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' that`s great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' shame!', ' happy', ' I like it!!', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']


In [7]:
# np.mean(check_corrs("roberta"),axis=1)

In [8]:
# np.mean([jaccard(x,y) for x,y in zip(all_preds["roberta_0"], test_df.selected_text)])

In [9]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large', do_lower_case=False)
model = RobertaForSentimentExtraction.from_pretrained('roberta-large', output_hidden_states=True)
model.cuda()
X_test, X_type_test = convert_lines(tokenizer, test_df, max_sequence_length= max_sequence_length)
seeds = [43,53,63]
for seed in seeds:
    for fold in range(5):
        model.load_state_dict(torch.load(f"./models/roberta-large_{fold}_{seed}.bin"), strict=False)
        model.eval()
        all_start_top_log_probs,all_start_top_index,all_end_top_log_probs,all_end_top_index = get_predictions(X_test, X_type_test, model)
        selected_texts = get_selected_texts(X_test, tokenizer)
        print(selected_texts[:10])
        all_preds[f"roberta-large_{fold}_{seed}"] = [x if type(x) is str else "" for x in selected_texts]


['Last session of the day', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' shame!', ' happy', ' I like it!!', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']


...


['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' shame!', ' happy', ' I like it!!', ' great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day  http://twitpic.com/67ezh', ' exciting', ' such a shame!', ' happy bday!', ' like', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']


In [10]:
# np.mean(check_corrs("roberta-large"),axis=1)

In [11]:
tokenizer = BartTokenizer.from_pretrained('bart-large', do_lower_case=False)
model = BartForSentimentExtraction.from_pretrained('bart-large', output_hidden_states=True)
model.cuda()
X_test, X_type_test = convert_lines(tokenizer, test_df, max_sequence_length= max_sequence_length)
seeds = [73,83,93]
for seed in seeds:
    for fold in range(5):
        model.load_state_dict(torch.load(f"./models/bart_{fold}_{seed}.bin"), strict=False)
        model.eval()
        all_start_top_log_probs,all_start_top_index,all_end_top_log_probs,all_end_top_index = get_predictions(X_test, X_type_test, model)
        selected_texts = get_selected_texts(X_test, tokenizer)
        print(selected_texts[:10])
        all_preds[f"bart_{fold}_{seed}"] = [x if type(x) is str else "" for x in selected_texts]


['Last session of the day', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!', ' HATES', ' completely blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' that`s great!', ' HATES ME', ' completely blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' really exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' really exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!', 'I THINK EVERYONE HATES ME ON HERE', ' completely blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' that`s great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']


...


['Last session of the day', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!', ' HATES', ' completely blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!', ' HATES', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' such a shame!', ' happy bday!', ' I like it!!', ' that`s great!', ' HATES ME', ' soooooo wish i could, but im in school and myspace is completely blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' that`s great!', 'I THINK EVERYONE HATES ME ON HERE', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']



['Last session of the day', ' exciting', ' shame!', ' happy bday!', ' I like it!!', ' that`s great!', 'I THINK EVERYONE HATES ME ON HERE', ' blocked', ' and within a short time of the last clue all of them', ' What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!']


In [12]:
# model_list = [f"bart_{i}" for i in range(5)]
# model_list = [f"roberta_{i}" for i in range(5)]
# model_list = [f"xlmr_{i}" for i in range(5)]
# model_list = [f"roberta-squad_{i}" for i in range(5)]
# model_list = [f"roberta_{i}" for i in range(5)]
# model_list = [f"roberta-large_{i}" for i in range(5)] + [f"roberta_{i}" for i in range(5)] +  [f"bart_{i}" for i in range(5)]
model_list = [key for key in all_preds.keys()]
print(model_list)

['roberta_0_13', 'roberta_1_13', 'roberta_2_13', 'roberta_3_13', 'roberta_4_13', 'roberta_0_23', 'roberta_1_23', 'roberta_2_23', 'roberta_3_23', 'roberta_4_23', 'roberta_0_33', 'roberta_1_33', 'roberta_2_33', 'roberta_3_33', 'roberta_4_33', 'roberta-large_0_43', 'roberta-large_1_43', 'roberta-large_2_43', 'roberta-large_3_43', 'roberta-large_4_43', 'roberta-large_0_53', 'roberta-large_1_53', 'roberta-large_2_53', 'roberta-large_3_53', 'roberta-large_4_53', 'roberta-large_0_63', 'roberta-large_1_63', 'roberta-large_2_63', 'roberta-large_3_63', 'roberta-large_4_63', 'bart_0_73', 'bart_1_73', 'bart_2_73', 'bart_3_73', 'bart_4_73', 'bart_0_83', 'bart_1_83', 'bart_2_83', 'bart_3_83', 'bart_4_83', 'bart_0_93', 'bart_1_93', 'bart_2_93', 'bart_3_93', 'bart_4_93']


In [13]:
# for model_name in model_list:
#     test_df[model_name] = all_preds[model_name]
# test_df["ensembled"] = test_df.progress_apply(lambda row: ensemble(row["sep_text"], [row[model_name] for model_name in model_list]),axis=1)

In [16]:
all_vals = [all_preds[model_name] for model_name in model_list]
print(len(all_vals))
ensembled = []
sep_texts = test_df.text.values
# sep_texts = test_df.text.apply(lambda x: " ".join(x.split()).lower())

for i in tqdm(range(len(test_df))):
    predictions = [val[i] for val in all_vals]
    ensembled.append(ensemble(sep_texts[i], predictions))

45


In [ ]:
# for i in range(5):
#     print(i, np.mean([jaccard(x,y) for x,y in zip(test_df.selected_text, all_preds[f"roberta_{i}"])]))
# print(np.mean([jaccard(x,y) for x,y in zip(test_df.selected_text, ensembled)]))

In [ ]:
# scores1 = []
# scores2 = []
# scores3 = []
# for x,y,z in zip(test_df.text, ensembled, test_df.selected_text):
#     if "   " in x:
# #         print(x,"|,",y,"|",z)
#         scores1.append(jaccard(y,z))
#         start,end = x.index(y), x.index(y) + len(y)
#         if x.index("   ") < start and start :
#             print(x,"|,",y,"|",z)
#             scores3.append(jaccard(x[start-1:end-1],z))
#             scores2.append(jaccard(x[start-1:end-1],z))
#         else:
#             scores3.append(jaccard(y,z))
#             scores2.append(jaccard(y,z))
#     else:
#         scores2.append(jaccard(y,z))

# print(np.mean(scores3), np.mean(scores1),np.mean(scores2))

In [ ]:
# fixed = []
# for x,y in zip(test_df.text, ensembled):
#     if "   " in x:
#         start,end = x.index(y), x.index(y) + len(y)
#         if x.index("   ") < start:
#             fixed.append(x[start-1:end-1])
#         else:
#             fixed.append(y)
#     else:
#         fixed.append(y)

In [19]:
test_df["selected_text"] = ensembled
test_df[["textID","selected_text"]].to_csv("sub_all.csv",index=False)
# test_df["selected_text"] = fixed
# test_df[["textID","selected_text"]].to_csv("sub_pp.csv",index=False)

In [ ]:
# test_df

In [ ]:
# test_df.to_csv("test.csv",index=False)